In [ ]:
import dotenv
import os
dotenv.load_dotenv('./Mehmet2/.env')

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./Mehmet/trials/trial_01.txt")
document = loader.load()

In [ ]:
chunks = [text.strip() for text in document[0].page_content.split('|')]
display(chunks)

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.environ["OPENAI_API_KEY"],
    api_base=os.environ["OPENAI_API_BASE_URL"],
    api_type="azure",
    deployment_id="embeddings",
    api_version=os.environ["OPENAI_API_VERSION"],
    model_name="text-embedding-ada-002",
)

In [24]:
import chromadb

client = chromadb.Client()

In [25]:
client.delete_collection('trial-01')

In [26]:
collection = client.create_collection(
    "trial-01", metadata={"hnsw:space": "ip"}, embedding_function=openai_ef, get_or_create=True
)

In [27]:
collection.add(
    documents=chunks,
    ids=[f'id{i}' for i in range(len(chunks))]
)

In [34]:
import tiktoken

def count_message_tokens(messages: list[str]):
        """See [Azure OpenAI Samples](https://github.com/Azure/openai-samples/blob/main/Basic_Samples/Chat/chatGPT_managing_conversation.ipynb)"""
        encoding = tiktoken.encoding_for_model('gpt-4')
        num_tokens = 0
        for message in messages:
            num_tokens += len(encoding.encode(message))
        return num_tokens

In [35]:
(results_q1, ) = collection.query(
    query_texts=['Extract the cohort(s) from the protocol document.'],
)['documents']

count_message_tokens(results_q1)

225